In [26]:
import pandas as pd
import geopandas as gp
import numpy as np
import os
import itertools
import glob
from datetime import date

In [27]:
## initiate file names for output results
current_results_file = f"nov23_results_joined_SL_{date.today().strftime('%Y%m%d')}"
all_results_file = f"all_results_joined_SL_{date.today().strftime('%Y%m%d')}"

# initate folder paths for data to read in
folder_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12"
prev_results = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/all_results_joined_SL_20231214.csv"
sl_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/Master_Screening_Levels.xlsx"
pcb_arc_lookup_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/PCB_aroclor_lookup.csv"

PROCESS RESULTS SPREADSHEETS

In [14]:
file_extension = '*.xlsx'

# List all files with the specified extension in the folder
files = glob.glob(os.path.join(folder_path, file_extension))

results_df = []
length = 0
# Iterate through each file and read its content
for file_path in files:
    print(file_path)
    df = pd.read_excel(file_path)
    df.columns = df.columns.str.replace("_"," ")
    results_df.append(df)

results_df = pd.concat(results_df)
print(len(results_df))

/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/311115 SGS EDD.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/311175 SGS EDD Soil.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/311175 SGS EDD water.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/B8755_EDD.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/University of Washington Bothell Duwamish RCN 311018 EIM.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/University of Washington Bothell Duwamish RCN 311115 EIM.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/University of Washington Bothell UW (IAS) 311175 EIM.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Resu

In [15]:
# remove any rows that were not field data
results_df =  results_df[results_df['Field Collection Start Date'].isna() == False]
len(results_df)

1720

In [16]:
print(results_df['Sample Matrix'].unique())
print(results_df['Sample Source'].unique())

# create new column of sample type based on sample matrix and sample source columns
results_df['Sample Matrix_clean'] = np.where(results_df['Sample Matrix']=='Aqueous', 'Water', results_df['Sample Matrix'])
results_df['Sample Matrix_clean'] = np.where(results_df['Sample Matrix'].str.contains('Solid'), 'Soil', results_df['Sample Matrix_clean'])
results_df['Sample Matrix_clean'] = np.where(results_df['Sample Source']=='Groundwater', 'Water', results_df['Sample Matrix_clean'])

['Aqueous' 'Solid' 'Solid/Sediment' 'Water']
[nan 'Soil' 'Groundwater']


In [17]:
# TODO: come back and figure out the PCBs with multiple numbers and make sure they are the right chemical name
# remove leading PCB-### from result parameter name to match screening level spreadsheet

def clean_fun(val):
    if 'PCB' in val:
        if 'aroclor' not in val:
            if '...' in val:
                return 'FLAG'
            list = val.split(' ')
            if len(list)>1:
                return list[1]
            else:
                return 'FLAG'
    return val

results_df['Result Parameter Name'] = results_df['Result Parameter Name'].apply(lambda x: clean_fun(x))

In [18]:
# remove trailing spaces in sample id
results_df['Sample ID'] = results_df['Sample ID'].str.strip()

# change sample ID DPS1 -> DPS-1
results_df['Sample ID'] = np.where(results_df['Sample ID'] == 'DPS1', 'DPS-1', results_df['Sample ID'])

# change sample ID SPB-0159-S-1 -> SPB-O159-S-1
results_df['Sample ID'] = np.where(results_df['Sample ID'] == 'SPB-0159-S-1', 'SPB-O159-S-1', results_df['Sample ID'])

# replace typo from 11/17 -> 11/07
results_df['Field Collection Start Date'] = np.where(results_df['Field Collection Start Date']=='2023-11-17 00:00:00', np.datetime64('2023-11-07'), results_df['Field Collection Start Date'])

In [19]:
# remove unnecessary columns from raw data
results_df = results_df[['Sample ID','Field Collection Start Date','Sample Matrix','Sample Source','Result Parameter Name','Result Value',
       'Result Value Units', 'Result Reporting Limit',
       'Result Reporting Limit Type', 'Result Detection Limit',
       'Result Detection Limit Type', 'Result Data Qualifier']]

In [ ]:
# split results into soil and water
results_soil_df = results_df[results_df['Sample Matrix_clean']=='Soil']
results_water_df = results_df[results_df['Sample Matrix_clean']=='Water']

JOIN TABLES OF RESULTS TO MASTER SCREENING LEVELS FROM F&B

In [20]:
# create data frames of the raw data and the lookup
pcb_arc_lookup = pd.read_csv(pcb_arc_lookup_path)

# create data frame of the screening levels for soils and
sl_soil_df = pd.read_excel(sl_path, sheet_name='Soil')
sl_water_df = pd.read_excel(sl_path, sheet_name='Water')

# concatenate to all screening levels
sl = pd.concat([sl_soil_df, sl_water_df])

In [21]:
# strip dioxin furans screening levels of their commas to match the results spreadsheet
sl_soil_df['Chemical'] = np.where(sl_soil_df['Chemical Group']== 'Dioxin Furans', sl_soil_df['Chemical'].str.replace(',',''), sl_soil_df['Chemical'])
sl_water_df['Chemical'] = np.where(sl_water_df['Chemical Group']== 'Dioxin Furans', sl_water_df['Chemical'].str.replace(',',''), sl_water_df['Chemical'])

In [22]:
# strip pcbs of their commas to match the results spreadsheet
sl_soil_df['Chemical'] = np.where(sl_soil_df['Chemical Group']== 'PCB', sl_soil_df['Chemical'].str.replace(',',''), sl_soil_df['Chemical'])
sl_water_df['Chemical'] = np.where(sl_water_df['Chemical Group']== 'PCB', sl_water_df['Chemical'].str.replace(',',''), sl_water_df['Chemical'])

In [23]:
# format pcb to aroclor loo
pcb_arc_lookup = pcb_arc_lookup[pcb_arc_lookup['Aroclor Name'].str.contains('aroclor', na=False)][['PCB Isomer', 'Aroclor Name']]

JOIN SCREENING LEVELS TO RESULTS

In [24]:
# replace pcb names with the aroclor names to match with F&B results
sl_soil_df_join = pd.merge(sl_soil_df, pcb_arc_lookup, how = 'outer', left_on = 'Chemical', right_on = 'PCB Isomer')
sl_soil_df_join['Chemical'] = np.where(sl_soil_df_join['Aroclor Name'].str.contains('aroclor', na=False), sl_soil_df_join['Aroclor Name'], sl_soil_df_join['Chemical'])
sl_soil_df_join.drop(columns = pcb_arc_lookup.columns, inplace = True)
sl_soil_df_join = sl_soil_df_join[sl_soil_df_join['Medium']=='Soil']

In [25]:
# replace pcb names with the aroclor names to match with F&B results
sl_water_df_join = pd.merge(sl_water_df, pcb_arc_lookup, how = 'outer', left_on = 'Chemical', right_on = 'PCB Isomer')
sl_water_df_join['Chemical'] = np.where(sl_water_df_join['Aroclor Name'].str.contains('aroclor', na=False), sl_water_df_join['Aroclor Name'], sl_water_df_join['Chemical'])
sl_water_df_join.drop(columns = pcb_arc_lookup.columns, inplace = True)
sl_water_df_join = sl_water_df_join[sl_water_df_join['Medium']=='Water']

In [277]:
# join screening levels to the results
soil_sl_join = pd.merge(sl_soil_df_join,results_soil_df,how = 'outer', left_on = ['Chemical'], right_on = ['Result Parameter Name'])

In [278]:
# remove screening levels that do not have values
soil_sl_join = soil_sl_join[(soil_sl_join['Screening Level Measurement']!='na')]
soil_sl_join = soil_sl_join[(soil_sl_join['Screening Level Measurement']!='TBD')]
soil_sl_join = soil_sl_join[(soil_sl_join['Screening Level Measurement']!='PQL')]

In [279]:
# calculate whether the screening levels have been exceeded
soil_sl_join['SL_exceeded'] = np.where(soil_sl_join['Screening Level Measurement'] <soil_sl_join['Result Value'],'Y','N')

JOIN SCREENING LEVELS TO RESULTS - WATER

In [280]:
# join screening levels to results
water_sl_join = pd.merge(sl_water_df_join,results_water_df,how = 'outer', left_on = ['Chemical'], right_on = ['Result Parameter Name'])

In [281]:
water_sl_join = water_sl_join[(water_sl_join['Screening Level Measurement']!='na')]
water_sl_join = water_sl_join[(water_sl_join['Screening Level Measurement']!='TBD')]
water_sl_join = water_sl_join[(water_sl_join['Screening Level Measurement']!='PQL')]

In [282]:
# convert any screening levels that are in mg/L to ug/L
water_sl_join[water_sl_join['SL Unit'] == 'mg/L']

water_sl_join['Screening_Level_Measurement_Convert'] = np.where(water_sl_join['SL Unit'] == 'mg/L', water_sl_join['Screening Level Measurement']*1000, water_sl_join['Screening Level Measurement'])
water_sl_join['Screening_Level_Measurement_Convert'] = np.where(water_sl_join['SL Unit'] == 'ppm', water_sl_join['Screening Level Measurement']*1000, water_sl_join['Screening Level Measurement'])

In [283]:
# calculate whether the screening levels have been exceeded
water_sl_join['SL_exceeded'] = np.where(water_sl_join['Screening_Level_Measurement_Convert']< water_sl_join['Result Value'], 'Y','N')

In [284]:
columns = ['Medium', 'Sample ID','Field Collection Start Date', 'Chemical Group', 'Chemical', 'Result Parameter Name', 'Scenario', 'Screening Level Type',
       'Screening Level Measurement', 'Result Value', 'Result Data Qualifier', 'SL Unit', 'Source', 'SL_exceeded']

all_results = pd.concat([soil_sl_join[columns],water_sl_join[columns]])
all_results['SL_diff'] = all_results['Result Value'] - all_results['Screening Level Measurement']

In [285]:
# where the screening level is blank, replace exceedance with "no screening level identified"
all_results['Screening Level Measurement'].fillna('No Screening Level Identified', inplace = True)
all_results['SL_exceeded'] = np.where(all_results['Screening Level Measurement']=='No Screening Level Identified','No Screening Level Identified', all_results['SL_exceeded'])

In [286]:
all_results.dropna(subset=['Sample ID'], inplace=True)
all_results.rename(columns = {'Field Collection Start Date': 'DATE'}, inplace = True)
all_results.to_csv(f'/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/{current_results_file}.csv', index = False)

JOIN TO PREVIOUS SCREENING RESULTS

In [287]:
# join to previous results
prev_results_df= pd.read_csv(prev_results)

In [288]:
#all_results_df = pd.concat([prev_results_df, all_results])
all_results.to_csv(f'/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/{all_results_file}.csv', index = False)